# Packages

In [1]:
# Use autoreload to automatically reload modules
%load_ext autoreload
%autoreload 2

In [9]:
# Setup paths
import rootutils
import xarray as xr
import pandas as pd
root_path = rootutils.find_root()


In [3]:
def img_interactive(da):

    import ipywidgets as widgets
    import matplotlib.pyplot as plt

    def plot_time_step(time_index):
        plt.figure(figsize=(8, 8))
        da.isel(time=time_index).plot.imshow(robust=True)
        plt.title(f"{str(da.time.values[time_index])[:10]}")
        plt.axis('off')
        plt.show()

    widgets.interact(plot_time_step, time_index=widgets.IntSlider(min=0, max=len(da.time)-1, step=1, value=0))

In [ ]:
fire_df = pd.read_parquet(root_path / "data" / "raw" / "spain_fires_25.parquet")

Timestamp('2025-06-27 22:14:21')

In [66]:
i = 24

In [ ]:
print(fire_df["Time"].iloc[i])
da_s2 = xr.open_dataarray(root_path / "data" / "raw" / str(i) / "da_s2.nc" ) 
da_s2_rgb = da_s2.sel(band=["B04", "B03", "B02"])
img_interactive(da_s2_rgb)

2025-03-31 21:55:32


interactive(children=(IntSlider(value=0, description='time_index', max=25), Output()), _dom_classes=('widget-i…